In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [3]:
city_list = city_link_collector()
len(city_list)

415

In [4]:
def craigslist_motorcycle_scraper(city_list):
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts']
    for city in city_list: 

        #get the first page of the Austin motorcycle prices
        city_response = requests.get('{}search/mca?s=0&bundleDuplicates=1'.format(city))
        #parse through it and make it readable
        html_soup = BeautifulSoup(city_response.text, 'html.parser')
        #find the total number of posts to find the limit for each page
        results_num = html_soup.find('div', class_= 'search-legend')
        #pulled the total count of posts as the upper bound of the pages array
        results_total = int(results_num.find('span', class_='totalcount').text) 
        #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
        pages = np.arange(0, results_total+1, 120)
        print("{} Posts = {}".format(city, results_total))
        print("{} Pages = {}".format(city.title(), len(pages)))

        iterations = 0

        for page in pages:         
            
            #get request      
            post_response = requests.get("{}search/mca?".format(city) 
                           + "s=" #the parameter for defining the page number 
                           + str(page) #the page number in the pages array from earlier
                           + "&bundleDuplicates=1")

            time.sleep(random.randint(2,3))

            #throw warning for status codes that are not 200
            if post_response.status_code != 200:
                warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))

            #define the html text
            post_soup = BeautifulSoup(post_response.text, 'html.parser')

            count = 0
            for post in post_soup.find_all('a', class_ = 'result-title hdrlnk'):
                link = post['href']
                count += 1
                
#               checks to see if post html is loaded in our database and scrapes html if not
                if not (post_html.find_one({'_id': link}, {'html': requests.get(link).text})):
#                     print(str(link))
                    sub_post = requests.get(link)
                    post_html.insert_one({'_id': link, 'html': sub_post.text})
                    if sub_post.status_code != 200:
                        warn('Request: {}; Status code: {}'.format(requests, post_response.status_code))
                    time.sleep(random.randint(2,3)) #sleep timer to avoid being banned      
                else:
#                     print('Passing ' + str(link))
                    pass
                         
                if count == results_total:
                    break

            iterations += 1
            print("{} Page ".format(city.title()) + str(iterations) + " of {} pages".format(len(pages)) + " scraped successfully!")

            

        print("\n")
        print("{} complete!".format(city.title()))
        print('~' + str(len(pages)*120) + " rows collected.")
        print("\n")

In [6]:
client = MongoClient('localhost', 27017)
db = client['craigslist_motorcycles']
post_html = db['motorcycle_posts']

In [ ]:
craigslist_motorcycle_scraper(city_list[193:])

https://minneapolis.craigslist.org/ Posts = 1317
Https://Minneapolis.Craigslist.Org/ Pages = 11
Https://Minneapolis.Craigslist.Org/ Page 1 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 2 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 3 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 4 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 5 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 6 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 7 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 8 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 9 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 10 of 11 pages scraped successfully!
Https://Minneapolis.Craigslist.Org/ Page 11 of 11 pages scraped successfully!


Https://Minneapolis.Craigslist.Org/ complete!
~1320 r

Https://Northplatte.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Northplatte.Craigslist.Org/ complete!
~120 rows collected.


https://omaha.craigslist.org/ Posts = 287
Https://Omaha.Craigslist.Org/ Pages = 3
Https://Omaha.Craigslist.Org/ Page 1 of 3 pages scraped successfully!
Https://Omaha.Craigslist.Org/ Page 2 of 3 pages scraped successfully!
Https://Omaha.Craigslist.Org/ Page 3 of 3 pages scraped successfully!


Https://Omaha.Craigslist.Org/ complete!
~360 rows collected.


https://scottsbluff.craigslist.org/ Posts = 20
Https://Scottsbluff.Craigslist.Org/ Pages = 1
Https://Scottsbluff.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Scottsbluff.Craigslist.Org/ complete!
~120 rows collected.


https://elko.craigslist.org/ Posts = 2
Https://Elko.Craigslist.Org/ Pages = 1
Https://Elko.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Elko.Craigslist.Org/ complete!
~120 rows collected.


https://lasvegas.craigslist.org/ Posts = 517


https://newyork.craigslist.org/ Posts = 670
Https://Newyork.Craigslist.Org/ Pages = 6
Https://Newyork.Craigslist.Org/ Page 1 of 6 pages scraped successfully!
Https://Newyork.Craigslist.Org/ Page 2 of 6 pages scraped successfully!
Https://Newyork.Craigslist.Org/ Page 3 of 6 pages scraped successfully!
Https://Newyork.Craigslist.Org/ Page 4 of 6 pages scraped successfully!
Https://Newyork.Craigslist.Org/ Page 5 of 6 pages scraped successfully!
Https://Newyork.Craigslist.Org/ Page 6 of 6 pages scraped successfully!


Https://Newyork.Craigslist.Org/ complete!
~720 rows collected.


https://oneonta.craigslist.org/ Posts = 10
Https://Oneonta.Craigslist.Org/ Pages = 1
Https://Oneonta.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Oneonta.Craigslist.Org/ complete!
~120 rows collected.


https://plattsburgh.craigslist.org/ Posts = 55
Https://Plattsburgh.Craigslist.Org/ Pages = 1
Https://Plattsburgh.Craigslist.Org/ Page 1 of 1 pages scraped successfully!


Https://Plattsburgh.

https://cleveland.craigslist.org/ Posts = 396
Https://Cleveland.Craigslist.Org/ Pages = 4
Https://Cleveland.Craigslist.Org/ Page 1 of 4 pages scraped successfully!
Https://Cleveland.Craigslist.Org/ Page 2 of 4 pages scraped successfully!
Https://Cleveland.Craigslist.Org/ Page 3 of 4 pages scraped successfully!
Https://Cleveland.Craigslist.Org/ Page 4 of 4 pages scraped successfully!


Https://Cleveland.Craigslist.Org/ complete!
~480 rows collected.


https://columbus.craigslist.org/ Posts = 407
Https://Columbus.Craigslist.Org/ Pages = 4
Https://Columbus.Craigslist.Org/ Page 1 of 4 pages scraped successfully!
Https://Columbus.Craigslist.Org/ Page 2 of 4 pages scraped successfully!
Https://Columbus.Craigslist.Org/ Page 3 of 4 pages scraped successfully!
Https://Columbus.Craigslist.Org/ Page 4 of 4 pages scraped successfully!


Https://Columbus.Craigslist.Org/ complete!
~480 rows collected.


https://dayton.craigslist.org/ Posts = 91
Https://Dayton.Craigslist.Org/ Pages = 1
Https://Dayton

In [5]:
city_list[193]

'https://minneapolis.craigslist.org/'